In [1]:
from itertools import product
from math import sin, cos
from tqdm import tqdm
import numpy as np
from z3 import *

### Constantes globais

In [2]:
V_LOW, V_HIGH = 1, 10 # velocidades dos modos baixo e alto, em m
THETA = 15 # angulo de viragem entre modos
TAU = 1 # tempo minimo entre transicoes timed
ALL_ROUTES = [i*THETA for i in range(int(360/THETA))] # todos os ângulos possíveis
ALL_VELS = [V_LOW, V_HIGH] # todas as velocidades possiveis

GAUSS_DP = 80 # desvio padrão da gaussiana de inicialização das posições em m
SAFE_DIST = 50 # distância de segurança entre barcos em m

Mode, (M_LOW, M_HIGH) = EnumSort("Mode", ("V_LOW", "V_HIGH"))

### Funções auxiliares

In [3]:
deg_to_rad = lambda a: a * np.pi / 180
rad_to_deg = lambda a: a * 180 / np.pi
z3tofloat = lambda v: float(v.numerator_as_long())/float(v.denominator_as_long())
Abs = lambda x: If(x>=0, x, -x)
val_angle = lambda a: If(a<0, 360+a, If(a>=360, a-360, a))
vel_to_mode = lambda v: M_LOW if (v == V_LOW) else M_HIGH

## Funções de declaração, inicialização e transição

In [4]:
def declare(t, num_boats=3):
    # Iterar os barcos
    trace = {}
    for i in range(num_boats):
        trace[i] = {}
        
        # Variáveis de modo
        trace[i]["v"] = Const(f"b{i}_v{t}", Mode)
        trace[i]["a"] = Int(f"b{i}_a{t}")
        
        # Variáveis de estado
        trace[i]["x"] = Real(f"b{i}_x{t}")
        trace[i]["y"] = Real(f"b{i}_y{t}")
        trace[i]["t"] = Real(f"b{i}_t{t}")
        
    # Distancia de seguranca
    trace["d"] = Int(f"d{t}")
        
    return trace
        
        
def init(tr):
    # Gerar ângulos iniciais aleatórios
    random_angles = ALL_ROUTES
    
    # Iterar os barcos
    r = []
    for i in range(len(tr)-1):
        np.random.shuffle(random_angles)
        
        # Condições do modo
        r.append(tr[i]["v"] == M_HIGH)
        r.append(tr[i]["a"] == random_angles[0])
        
        # Condições do estado
        r.append(tr[i]["x"] == GAUSS_DP * np.random.randn())
        r.append(tr[i]["y"] == GAUSS_DP * np.random.randn())
        r.append(tr[i]["t"] == 0)
        
    # Condicao da distancia de seguranca
    r.append(And(tr["d"] > 0, tr["d"] < 100))
        
    r = And(r)
    
    return r


def switch_safe(tr, boat_id):
    r = []
    
    # Iterar cada um dos barcos
    for i in range(len(tr)-1):
        if i != boat_id:
            # Condição das distâncias
            x = Abs(tr[i]["x"]-tr[boat_id]["x"]) > tr["d"]
            y = Abs(tr[i]["y"]-tr[boat_id]["y"]) > tr["d"]
            
            t_conds = []
            all_vels = list(product(*[ALL_VELS, ALL_VELS]))
            for v1, v2 in all_vels:
                
                # Condição do tempo
                m1, m2 = vel_to_mode(v1), vel_to_mode(v2)
                t1 = And(m1 == tr[boat_id]["v"], m2 == tr[i]["v"])
                t2 = Abs(tr[i]["t"]-tr[boat_id]["t"])*(v1+v2)/2 > tr["d"]
                t_conds.append(And(t1, t2))
            
            t = Or(t_conds)
            
            # Verificar se uma das distâncias é superior à distância de segurança
            r.append(Or(x, y, t))
            
    r = And(r)
            
    return r
        
    
def timed(prev, curr):
    modes = list(product(*[ALL_VELS, ALL_ROUTES]))
    
    # Iterar cada um dos barcos
    r = []
    for i in range(len(prev)-1):
        
        # Condição dos modos
        r.append(curr[i]["v"] == prev[i]["v"])
        
        # Condição do tempo
        r.append(curr[i]["t"] - prev[i]["t"] > TAU)
        
        # Condição da rota e posição
        routes_cond = []
        for j in range(len(modes)):
            route_cond = []
            
            # Condição do ângulo
            v, a = modes[j]
            route_cond.append(prev[i]["a"] == a)
            route_cond.append(curr[i]["a"] == a)
            
            # Incremento de posição
            dx = v * cos(deg_to_rad(a)) * (curr[i]["t"] - prev[i]["t"])
            dy = v * sin(deg_to_rad(a)) * (curr[i]["t"] - prev[i]["t"])
            
            # Condição da posição
            route_cond.append(curr[i]["x"] == prev[i]["x"] + dx)
            route_cond.append(curr[i]["y"] == prev[i]["y"] + dy)
            
            # Fazer o And de todas as condições
            routes_cond.append(And(route_cond))
        
        # Adicionar uma de todas as rotas possíveis
        r.append(Or(routes_cond))
        
    r = And(r)
        
    return r


def untimed(prev, curr):
    r = []
    
    # Iterar cada um dos barcos
    for i in range(len(prev)-1):
        
        # Condições da posição e tempo
        r.append(curr[i]["x"] == prev[i]["x"])
        r.append(curr[i]["y"] == prev[i]["y"])
        r.append(curr[i]["t"] == prev[i]["t"])

        # barco V_LOW transita para V_LOW
        low_low = []
        low_low.append(And(prev[i]["v"] == M_LOW, curr[i]["v"] == M_LOW))
        low_low.append(Not(switch_safe(prev, i)))
        low_low.append(Or(curr[i]["a"] == val_angle(prev[i]["a"]+THETA), curr[i]["a"] == val_angle(prev[i]["a"]-THETA)))
        low_low = And(low_low)

        # barco V_LOW transita para barco V_HIGH
        low_high = []
        low_high.append(And(prev[i]["v"] == M_LOW, curr[i]["v"] == M_HIGH))
        low_high.append(switch_safe(prev, i))
        low_high.append(curr[i]["a"] == prev[i]["a"])
        low_high = And(low_high)

        # barco V_HIGH transita para barco V_LOW
        high_low = []
        high_low.append(And(prev[i]["v"] == M_HIGH, curr[i]["v"] == M_LOW))
        high_low.append(Not(switch_safe(prev, i)))
        high_low.append(Or(curr[i]["a"] == val_angle(prev[i]["a"]+THETA), curr[i]["a"] == val_angle(prev[i]["a"]-THETA)))
        high_low = And(high_low)

        # barco V_HIGH transita para barco V_HIGH
        high_high = []
        high_high.append(And(prev[i]["v"] == M_HIGH, curr[i]["v"] == M_HIGH))
        high_high.append(switch_safe(prev, i))
        high_high.append(curr[i]["a"] == prev[i]["a"])
        high_high = And(high_high)

        # Adicionar uma destas possíveis transições
        r.append(Or(low_low, low_high, high_low, high_high))
        
    # O estado não pode ficar igual
    same = []
    for i in range(len(prev)-1):
        same.append(prev[i]["v"] == curr[i]["v"])
        same.append(prev[i]["a"] == curr[i]["a"])
        same.append(prev[i]["x"] == curr[i]["x"])
        same.append(prev[i]["y"] == curr[i]["y"])
        # same.append(prev[i]["t"] == curr[i]["t"])
    same = Not(And(same))

    # Todas as condições da transição devem ser cumpridas
    r = And(And(r), same)
    
    return r


def trans(prev, curr):
    # Condições timed e untimed
    untimed_cond = untimed(prev, curr)
    timed_cond = timed(prev, curr)
    
    # Condições de sincronismo
    eq_cond = And([curr[i]["t"] == curr[i+1]["t"] for i in range(len(curr)-2)])
    
    # Condicao da manutencao da distancia de seguranca
    d_cond = prev["d"] == curr["d"]
    
    r = And(Or(untimed_cond, timed_cond), eq_cond, d_cond)
    
    return r

In [5]:
def gen_trace(declare, init, trans, k):
    solver = Solver()
    trace = {i: declare(i) for i in range(k)}
    solver.add(init(trace[0]))
    
    for i in range(k-1):
        solver.add(trans(trace[i], trace[i+1]))
        
    if solver.check() == sat:
        m = solver.model()
        
        for i in range(k):
            print(f"\nTrace {i}:")
            for b in range(len(trace[i])-1):
                for v in trace[i][b]:
                    if trace[i][b][v].sort() == RealSort():
                        print(f"  {v}{b} = {z3tofloat(m[trace[i][b][v]])}")
                    else:
                        print(f"  {v}{b} = {m[trace[i][b][v]]}")
                print("")
            d = trace[i]["d"]
            print(f"  {d} = {m[d]}")
            print("")
                
        r = m
    else:
        r = None
        
    return r

m = gen_trace(declare, init, trans, 10)


Trace 0:
  v0 = V_HIGH
  a0 = 30
  x0 = 14.465666316545711
  y0 = -44.96681344577797
  t0 = 0.0

  v1 = V_HIGH
  a1 = 75
  x1 = -72.66938117951979
  y1 = 124.1692892888988
  t1 = 0.0

  v2 = V_HIGH
  a2 = 135
  x2 = -37.9935109341242
  y2 = 3.653847610245654
  t2 = 0.0

  d0 = 99


Trace 1:
  v0 = V_LOW
  a0 = 45
  x0 = 14.465666316545711
  y0 = -44.96681344577797
  t0 = 0.0

  v1 = V_HIGH
  a1 = 75
  x1 = -72.66938117951979
  y1 = 124.1692892888988
  t1 = 0.0

  v2 = V_LOW
  a2 = 120
  x2 = -37.9935109341242
  y2 = 3.653847610245654
  t2 = 0.0

  d1 = 99


Trace 2:
  v0 = V_LOW
  a0 = 45
  x0 = 17.49509671069038
  y0 = -41.937383051633304
  t0 = 4.284261549664661

  v1 = V_HIGH
  a1 = 75
  x1 = -71.56053269626614
  y1 = 128.30756816629713
  t1 = 4.284261549664661

  v2 = V_LOW
  a2 = 120
  x2 = -59.4148186824475
  y2 = 40.75664099491049
  t2 = 4.284261549664661

  d2 = 99


Trace 3:
  v0 = V_LOW
  a0 = 30
  x0 = 17.49509671069038
  y0 = -41.937383051633304
  t0 = 4.284261549664661

 